In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [2]:
from keras import models
from keras import layers
from keras import optimizers

In [3]:
import matplotlib.pyplot as plt

In [4]:
import keras

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import numpy as np

In [7]:
num_classes = 2

In [8]:
vgg_conv = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(224, 224, 3))

In [9]:
layer_dict = dict([(layer.name, layer) for layer in vgg_conv.layers])

In [10]:
vgg_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
x = layer_dict['block5_pool'].output

In [12]:
x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation='softmax')(x)

In [13]:
custom_model = models.Model(input=vgg_conv.input, output=x)

/home/axreldable/anaconda3/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [14]:
# Make sure that the pre-trained bottom layers are not trainable
for layer in custom_model.layers[:19]:
    layer.trainable = False

In [15]:
# Do not forget to compile it
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy'])

In [16]:
import glob

In [17]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [ ]:
images = []
labels = []

In [ ]:
for fn in glob.glob('/home/spasenov/gate_dir/datasets/caltech/009.bear/*.jpg'):
    oriimg = cv2.imread(fn)
    image = cv2.resize(oriimg,(224,224),interpolation=cv2.INTER_AREA)
    images.append(image)
    labels.append(0)

In [ ]:
for fn in glob.glob('/home/spasenov/gate_dir/datasets/caltech/213.teddy-bear/*.jpg'):
    oriimg = cv2.imread(fn)
    image = cv2.resize(oriimg,(224,224),interpolation=cv2.INTER_AREA)
    images.append(image)
    labels.append(1)

In [ ]:
images = np.array(images)

In [ ]:
labels = np.array(labels)

In [ ]:
X_train_imgs, X_test_imgs, y_train, y_test = train_test_split(images, labels, test_size=0.33, random_state=15)

In [ ]:
X_train = X_train_imgs / 255.0
X_test = X_test_imgs / 255.0

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
history = custom_model.fit(X_train, y_train,
          batch_size=8,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
preds = custom_model.predict(X_test)

In [ ]:
pred_labels = np.argmax(preds, axis=1)

In [ ]:
for i in range(10):
    if pred_labels[i] == 0:
        print('bear')
    else:
        print('teddy-bear')
    plt.imshow(X_test[i])
    plt.show()